In [ ]:
#Load Data
import pandas as pd
import numpy as np
 
np.random.seed(8675309)
 
eider.s3.download('s3://eider-datasets/mlu/DATA_Training.csv','/tmp/DATA_Training.csv')
eider.s3.download('s3://eider-datasets/mlu/DATA_Public_Test.csv','/tmp/DATA_Public_Test.csv')
train = pd.read_csv('/tmp/DATA_Training.csv', na_values = 'null')
public_test = pd.read_csv('/tmp/DATA_Public_Test.csv', na_values = 'null')

Let's start with the variables that are most clearly numerical: the scores. In a certain way, imputation of missing data is a ML problem on its own: it is very similar to predicting unobserved star ratings. As with predicting star ratings, it is often a good idea to start with the simplest prediction method: filling in with the mean. For each score column, replace any missing value with the mean of the column. There are many way to do so, but I recommend directly using Pandas fillna. Remember to do the exact same transformation to the testing data features so that we fill in all the missing data in the same way for both training and prediction.

Note: If you are using fillna, the documentation doesn't make it very obvious, but you can fill only a specific set of columns by passing fillna a dataframe with columns labeled with the columns you want to fill, and the values the values you want to fill it with. For instance, passing train[['score1','score2']].mean() to fillna will fill just those two scores with their mean value.

In [ ]:

score_columns = ['score1','score2','score3','score4','score5']
# For each score column, Use pandas fillan to replace missing value with the mean of the column.
train.fillna(train[score_columns].mean(), inplace = True)
public_test.fillna(train[score_columns].mean(), inplace = True)
train.head(5)

Let us now turn to the categorical variables. The issue labels (CIL1-CIL4, and IL1-IL4) specify the type of issue that the customer encountered. Explain why it does not make sense to attempt to impute a variable here, but rather fill in with some default value like 'Not Specified'. The one remaining variable with missing data is contact_type, which as you likely identified last week, is an oddly encoded variable. It only takes two numerical values: 0.0, and 4.0. What type of imputation do you think is best suited here?

Implement the imputation in all of these cases.

Answer 2
QUESTION 1: FOR CIL1-CIL4, AND IL1-IL4: EXPLAIN WHY IT DOES NOT MAKE SENSE TO ATTEMPT TO IMPUTE A VARIABLE HERE, BUT RATHER FILL IN WITH SOME DEFAULT VALUE LIKE 'NOT SPECIFIED'.
As we know, the issue labels (CIL1-CIL4, and IL1-IL4) specify the type of issue that the customer encountered. They are nominal categorical features, a predicted value does not help here and may bring noisy to the other categories. For example, if we fill all missing value with the most frequent value, we may end up with an suboptimal model suffers from those predicted values, so it doesn’t make sense to try and predict a missing value in our case.
By using 'Not Specified', it is useful as it gives the possibility to group missing values as a separate category represented by a constant value.
QUESTION 2: FOR CONTACT_TYPE: IT ONLY TAKES TWO NUMERICAL VALUES: 0.0, AND 4.0. WHAT TYPE OF IMPUTATION DO YOU THINK IS BEST SUITED HERE?
contact_type is a categorical data even it's value is numerical
Count of train['contact_type'] == 4.0 is 394, accounts for 4.72% of overall records
Count of train['contact_type'] == 0.0 is 2742, accounts for 32.85% of overall records
We need some domain knowledge here, if contact_type=0.0 means this record generated through a phone call between customer and customer service, then the missing value has a great possiblity to be 0.0.
Option one: Using the most common value to fill the missing values
As we see, value 0.0 is the most common value of contact_type(33% vs 4.7%), so one possible imputation is use the most common value to fill the missing values.
Since the portion of missing value of contact_type is 66.3%, if we fill them with 0.0, the downside is our model may ignore the contact_type=4.0 in the end.
Option two: Using IterativeImputer to fill the missing values -A more sophisticated approach is to use the IterativeImputer class, if contact_type corelates to other features, we can use IterativeImputer to learn it and predict missing values.
In this case, we will use fillna to fill most common value 0.0 for missing contact_type.

In [ ]:
cil_columns_values = {'CIL1' : 'Not Specified', 'CIL2' : 'Not Specified', 'CIL3' : 'Not Specified', 'CLI4' : 'Not Specified', 'IL1' : 'Not Specified', 'IL2' : 'Not Specified', 'IL3' : 'Not Specified', 'IL4' : 'Not Specified'}
contact_type_value = {'contact_type' : 0.0}
 
contact_type_4 = train[train['contact_type'] == 4.0]['contact_type'].count()
contact_type_0 = train[train['contact_type'] == 0.0]['contact_type'].count()
contact_type_all = train['ID'].count()
print("Count of train['contact_type'] == 4.0 is {}, accounts for {:.2f}% of overall records".format(contact_type_4, 100 * contact_type_4/contact_type_all))
print("Count of train['contact_type'] == 0.0 is {}, accounts for {:.2f}% of overall records".format(contact_type_0, 100 * contact_type_0/contact_type_all))
 
# FOR CIL1-CIL4, AND IL1-IL4, fill in with 'Not Specified'
train.fillna(cil_columns_values, inplace = True) 
public_test.fillna(cil_columns_values, inplace = True) 
# FOR contact_type, fill in with most common value 0.0
train.fillna(contact_type_value, inplace = True) 
public_test.fillna(contact_type_value, inplace = True) 
train.head(5)

Using either Pandas get_dummies, or sklearn OneHotEncoder, one-hot encode the issue label columns and the device column of the dataset. Be very careful to apply the same encoding to both the test and train features. You may find it helpful to construct lists like

unique_elements = pd.concat([train['IL1'],public_test['IL1']]).unique().tolist()
which contains all unique choices that exist in the train and test. If you want to then tell Pandas what the possible values in a categorical column is, you can do so with a snippet like:

train['IL1'] = train['IL1'].astype('category').cat.set_categories(unique_elements)
test['IL1'] = test['IL1'].astype('category').cat.set_categories(unique_elements)
Once Pandas knows the possible values of the column, get_dummies will work as you desire and create the same columns on the test and train set.

I recommend you store these results in a separate dataframe rather than modifying the original one. As you will try other encodings later.

In [ ]:

 
issue_features = ['CIL1', 'CIL2', 'CIL3', 'CLI4', 'IL1', 'IL2', 'IL3', 'IL4','device']
 
# Try to collect all possible values of each column
for feature in issue_features:
    unique_elements = pd.concat([train[feature], public_test[feature]]).unique().tolist()
    train[feature] = train[feature].astype('category').cat.set_categories(unique_elements)
    public_test[feature] = public_test[feature].astype('category').cat.set_categories(unique_elements)
    
# Using a separate dataframe rather than modifying the original one. As you will try other encodings later.
public_test_with_dummies = pd.get_dummies(public_test)
train_with_dummies = pd.get_dummies(train)
 
#public_test_with_dummies.head(5)
train_with_dummies.head(5)

Now that this is out of the way, we are almost ready to train a model. This week, we'll be trying a logistic regression again, which is one of the machine learning techniques that is very sensitive to the scale of input variables. Use StandardScaler to standardize (make mean zero, variance one) the feature columns. Train a logistic regression model. We will discuss regularization of logistic regression somewhat in later weeks, but for now, just know there is a parameter C associated to the model which controls how the model attempts to limit overfitting. If the model seems to be overfitting, make C smaller, if it is underfitting, make C larger. Given the number of data columns we have, overfitting is likely.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')
sns.set()
 
X_train, X_test, y_train, y_test = train_test_split(train_with_dummies.drop('response',axis=1), 
                                                    train['response'], test_size=0.20, 
                                                    random_state=101)
 
#Use StandardScaler with( mean zero, variance one)
scaler = StandardScaler(copy=True, with_mean=False, with_std=False)
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)
scaled_public_test = scaler.transform(public_test_with_dummies)
 
# Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
#C_range = [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 50, 100]
C_range_micro = np.arange(0.0001, 0.01, 0.0001)
C_range_small = np.arange(0.01, 1, 0.01)
C_range_large = np.arange(1, 100, 1)
 
def tuneC(C_range, title):
    test_accuracy_array = pd.DataFrame(columns = ['C_values','AccuracyScore'])
    test_accuracy_array['C_values'] = C_range
    # Let's tune C a bit
    for i in range(len(C_range)):
        # Fit logistic regression model to training data
        clf = LogisticRegression(penalty = 'l2', C = C_range[i], random_state = 0)
        clf.fit(scaled_X_train, y_train)
        # Predicting
        y_test_predicted = clf.predict(scaled_X_test)
        acc_score = accuracy_score(y_test, y_test_predicted)
        # Saving accuracy score and F1Score in table
        test_accuracy_array.iloc[i, 1] = acc_score
        #Uncomment the follow print to get details
        #print("when C = {}, accuracy_score = {}".format(C_range[i], acc_score))
    plt.figure(figsize=(10, 10))
    plt.title(title)
    plt.xlabel('C_values')
    plt.ylabel('AccuracyScore')
    plt.plot(test_accuracy_array['C_values'], test_accuracy_array['AccuracyScore'])
    plt.show()
 
tuneC(C_range_micro, "Tune C from 0.0001 to 0.01")
tuneC(C_range_small, "Tune C from 0.01 to 1")
tuneC(C_range_large, "Tune C from 1 to 100")

Now for the open-ended part. We didn't do much feature engineering, and our choice of one-hot encoding all of the issue labels is naive at best, and could even be down-right harmful as it introduced many highly-correlated columns! Take some time now to engineer some better features, here is where you can get creative and really think about the problem. Here are a few questions to get you started (don't need to address these explicitly, just to get you thinking):

How can you try to reduce the dimensionality of the one-hot encoded features? Are there natural groupings that might be relevant for this problem that you can identify by hand? Perhaps unsupervised learning can be used to find useful clusters automatically?
Is the current encoding of day and hour reasonable? Would a one-hot encoding instead be reasonable? If neither are, what else can you think of trying?
Should more be done to normalize the scores, as currently the distributions are far from Gaussian?
Would other models (say a decision tree, or random forest) be a better choice?
While not strictly needed, remember that Pipeline helps keep our code organized. Here is a rather nice blog post that can be helpful to look at.

Next week we'll introduce a number of additional techniques that will help with additional feature engineering, and sklearn tools that will help with searching for the best sets of parameters, but for now just think and explore!

Q: How can you try to reduce the dimensionality of the one-hot encoded features? Are there natural groupings that might be relevant for this problem that you can identify by hand? Perhaps unsupervised learning can be used to find useful clusters automatically?
From small size of datasets, we can manually check the one-hot encoded features, and merge two highly corelated features(A and B are 99% similar), drop features only a few 1s(like only 0.1% of A has 1s), etc.
We can also consider use Hash trick.
In addition, Unsupervised dimensionality reduction definitely will help to reduce the dimensionality.
Q: Is the current encoding of day and hour reasonable? Would a one-hot encoding instead be reasonable? If neither are, what else can you think of trying?
Current encoding for day and hour are not reasonable, since the order day/hour of week does not help a lot in our model. E.g Thursday or Wednesday does not matter, 9:00AM or 10:0AM does not matter.
A one-hot encoding might be not reasonable as well, since it will increase the dimensionality 7times for Day, 24times for hour but will bring a notable benifit.
Group values into meaningfull category will help: For day, we can group them into weekday(0,4) and weekend(5, 6), and for hour, we can group them into morning, afternoon, evening, midnight, etc.
Q: Should more be done to normalize the scores, as currently the distributions are far from Gaussian?
we need to deal with the outliers.
Try to drop the outlier scores.
Impute the outlier scores by using mean, median, mode, etc.
Apply a transformer outlier scores.
Q: Would other models (say a decision tree, or random forest) be a better choice?
Maybe, LogisticRegression and decision tree(or random forest) are different in the way they generate decision boundaries.
For higher-dimensional datasets, trees may perform better than LogisticRegression, but it really depends on the data, we need to do a deep dive before we make the decision.
In this case, we should give a try on decision tree and random forest, and definitely need a while to tune the models.

In [ ]:
	
# Using Pipeline to help us keep our code organized.
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
sns.set()
 
# Build pipe with StandardScaler and LogisticRegression
clf = LogisticRegression(penalty = 'l2', random_state = 0)
scaler = StandardScaler(copy=True, with_mean=False, with_std=False)
pipe = Pipeline(steps=[('scaler', scaler), ('clf', clf)])
 
# let's tune C parameter range from 0.1 to 1 with step=0.1
param_grid = {
    'clf__C': np.arange(0.1, 1, 0.1),
}
# Try to search the best C parameter with cross validation
search = GridSearchCV(pipe, param_grid, iid = False, cv = 2)
search.fit(X_train, y_train)
print("Tune C parameter range from 0.1 to 1 with step=0.1")
print("Best parameter C : CV score = {:.3f} when {}:".format(search.best_score_, search.best_params_))
 
y_predict = search.predict(X_test)
 
plt.title("compare y_test vs y_predict with jitter and heatmap")
plt.xlabel("y_test")
plt.xlabel("y_predict")
#Plot predict values vs labels with jitter and heatmap
sns.regplot( x = y_test.values, y = y_predict, fit_reg = False,
           x_jitter = 0.05, y_jitter = 0.05, scatter_kws = {'alpha' : 1/3})
plt.show()
 
# Print confusion matrix
cm = confusion_matrix(y_test.values, y_predict)
print("Confusion matrix of y_test and y_predict when {}:".format(search.best_params_))
print(cm)